# PDO Final Project: Kayak Boat Hull Optimization


## Executive Summary
Designing a boat hull can come with a broad range of design constaints and requirements depending on the desired use. One such desired outcome of a boat hull design might be the stability of the boat as it takes a wave input from one side or another. While some oscillation is inevitable, we can design a boat hull that has the minimum resulting oscillation from an input wave.  
    
This design study takes a simple model of boat hull design and extends it to answer questions about the hull shape's response to a wave input. To do this, I considered the boat to be a simple system with a gravitational force and a buoyant force. As a starting point, I built off of a model by Zach del Rosario which evaluated the torque due to buoyancy as the boat rotates around its center of mass on flat water. I extended this model to evaluate the net torque on the boat due to both the angle of the water *and* the angle of the boat. This allowed me to simulate an environment where a stationary boat is subject to an arbitrary wave input, described by the angle of the water surrounding the boat. Specifically, my goal was to *minimize* the resulting angle of the boat hull after it was subject to this wave input.  

        
## Background
Boat users come in all breeds. Wakeboarders, fisherman, commuters, supply chain analysts, all rely on boats for one reason or another. Kayaks are one kind of boat which are used commonly in recreation, in which an individual sits in
: A multimedia (text/equations/figures as necessary) description of your project context, including project stakeholders and their needs.

## Formulation
: A multimedia (text/equations/figures as necessary) description of your project analysis. Must include a formal statement of your optimization problem, details on your model (both functions and distribution) and how you built this model, your optimization strategy (multi-objective? Monte Carlo or FORM? etc.), and method validation results (check for multiple minima, calibration of uncertainty propagation, etc.).
            Note that this section must present any and all limitations of your model (functions and distributions), particularly in light of how they affect your Results.
        Results: A multimedia (text/equations/figures as necessary) presentation of your results, including observations and analysis of those results in light of your project Background.

    Conclusion: A multimedia (text/equations/figures as necessary) description tying specific details from the Results section back to your stakeholder needs (see Background). You must answer the following questions:

        “What (if any) insights did you learn about designing for your chosen context and stakeholders?”

        “If you did not gain any insights, what future work would be necessary to gain insight into your problem?”
        “If you did gain insights, what design decisions would you be inclined to make, based on your current understanding?


In [1]:
import grama as gr
import pandas as pd

from boat_utils_new import *
from plotnine import *
import matplotlib.pyplot as plt
%matplotlib inline

DF = gr.Intention()

import numpy as np
from grama.fit import ft_gp, fit_gp
from sklearn.gaussian_process.kernels import ConstantKernel, RBF

In [2]:
def angle_data(X, df_hull, df_mass):
    '''
    X (iterable): b_l, b_h, b_num, w_l, w_h, w_num = X
    '''
    b_l, b_h, b_num, w_l, w_h, w_num = X
    
    df_res = pd.DataFrame()
    
    for water_angle in np.linspace(w_l, w_h, w_num):
        
        df_temp = (
            get_moment_curve(df_hull, df_mass, water_angle, a_l = b_l, a_h = b_h, num = b_num)
            >> gr.tf_select(
                'boat_angle',
                'M_net'
            ) 
            >> gr.tf_mutate(
                water_angle = water_angle,
                
            )
        )

        df_res = pd.concat((df_temp, df_res), axis = 0)
        
    
    df_res.reset_index(inplace=True, drop=True)
    return df_res

In [3]:
def angle_gauss(X, df_hull, df_mass):
    '''
    X (iterable): [b_l, b_h, b_num, w_l, w_h, w_num] = X
    '''
    b_l, b_h, b_num, w_l, w_h, w_num = X
    
    df_angle = angle_data(X, df_hull, df_mass)

    md_moment = (
        df_angle
        >> ft_gp(var=["boat_angle", "water_angle"], out=["M_net"], kernels=RBF(.08, length_scale_bounds="fixed"))
    )
    
    return df_angle, md_moment

### Moment of inertia

In [4]:
def find_J(df_hull, df_mass):
    '''
    '''
    # Find spatial vector between mass points and center of mass
    distances = df_hull[["x", "y"]].values - df_mass[["x", "y"]].values

    # calculate moment of intertia assuming uniform mass
    mag_squared = []
    for distance in distances:
        mag_squared.append( (distance[0]**2 + distance[1]**2) )
        
    s = np.sum(mag_squared)
    J = s * df_mass.mass[0]
    return J
    

## Simulate

In [25]:
def w_func(t):
    '''
    Define the input wave
    '''
# This wave was found to be reasonably stable for the boat parameters I am using
    w_out = (1 / (1 + (t - 1)**2) - 0.5) * 0.6
    if w_out > 0:
        return w_out
    else: 
        return 0

def update(times, theta_w, theta_b, theta_b_dot, theta_b_ddot, moment, md_moment, J):
    '''
    Make sure every list is updated
    '''
    times.append(times[-1] + t_step)

# Get input
    new_water_input = w_func(times[-1])
    theta_w.append(new_water_input)
    
# Get righting moment
    new_moment = (
            md_moment
            >> gr.ev_df(
                gr.df_make(
                    boat_angle = theta_b[-1],
                    water_angle = theta_w[-1]
                )
            )
            >> gr.tf_mutate(M_net = DF.M_net_mean)
            >> gr.tf_select("M_net")
        ).values[0][0]
    
#     print()
    net_moment = new_moment - b*theta_b_dot[-1] 
    
    moment.append(net_moment)
    
# Get acceleration
#     torque = -moment[-1] - b*theta_b_dot[-1] 

    new_acceleration = moment[-1]/J
    theta_b_ddot.append(new_acceleration)

# Get velocity
#     new_velocity = (theta_b_ddot[-1] - theta_b_ddot[-2]) * t_step + theta_b_dot[-1]
#     new_velocity = (theta_b_ddot[-1]) * t_step + theta_b_dot[-1]
    new_velocity = (theta_b_ddot[-1] + theta_b_ddot[-2]) / 2 * t_step + theta_b_dot[-1]
    
    theta_b_dot.append(new_velocity)

# Get position
#     new_position = (theta_b_dot[-1] - theta_b_dot[-2]) * t_step + theta_b[-1]
#     new_position = (theta_b_dot[-1]) * t_step + theta_b[-1]
    new_position = (theta_b_dot[-1] + theta_b_dot[-2])/2 * t_step + theta_b[-1]
    theta_b.append(new_position)
    
    
    lists = times, theta_w, theta_b, theta_b_dot, theta_b_ddot, moment, md_moment, J
    return(lists)

def run_sim(times, theta_w, theta_b, theta_b_dot, theta_b_ddot, moment, md_moment, J):
    while times[-1] < t_run - 2*t_step:
        update(times, theta_w, theta_b, theta_b_dot, theta_b_ddot, moment, md_moment, J)
    
    df_sim = gr.df_make(
        times = times, 
        theta_w = theta_w, 
        theta_b = theta_b, 
        theta_b_dot = theta_b_dot, 
        theta_b_ddot = theta_b_ddot, 
        moment = moment
    )
    
    df_sim = df_sim.set_index('times')
    return df_sim

def get_max_angle(df):
    '''
    Find the maximum angle determined by the boat as it rotates
    '''
    return df['theta_b'].max()

In [26]:
def kaboodle(n):
    '''
    take in a hull parameter
    output a maximum angle with the given wave input
    '''
# Initial conditions
    times = [0, t_step]
    theta_w = [0, 0]
    theta_b = [0, 0]
    theta_b_dot = [0, 0]
    theta_b_ddot = [0, 0]
    moment = [0,0]

# Hull parameters
    H = 2 # Height of boat
    W = 3.5 # Width of boat
#     n = n # Shape parameter
    d = .5 # Displacement ratio

    X = [H, W, n, d]

    df_hull, df_mass = make_hull(X)

# Moment of inertia of new hull
    J = find_J(df_hull, df_mass)

# Angle parameters
    b_h = np.pi/2
    b_l = -b_h
    b_num = 10

    w_h = np.pi/6
    w_l = -w_h
    w_num = 3

    Y = [b_l, b_h, b_num, w_l, w_h, w_num]

    df_angle, md_moment = angle_gauss(Y, df_hull, df_mass)
    
    df_sim = run_sim(times, theta_w, theta_b, theta_b_dot, theta_b_ddot, moment, md_moment, J)
    max_angle = get_max_angle(df_sim)
    
    return max_angle

In [27]:
# Time parameters for simulation
t_run = np.pi*4 # [s]
t_step = 0.01 # [s]

# Damping ratio
b = 1

In [29]:
angles = []
for n in np.linspace(0.5, 1.5, 6):
    new = kaboodle(n)
    print(new)
    angles.append(new)

angles

0.024249030749918705
0.021686846221204644
0.019876643849168124
0.01822225479999305
0.016580669329502837
0.015952727475426743


[0.024249030749918705,
 0.021686846221204644,
 0.019876643849168124,
 0.01822225479999305,
 0.016580669329502837,
 0.015952727475426743]

In [11]:
md_minimize = (
    gr.Model()
    >> gr.cp_function(
        fun = lambda X: kaboodle(X[0]),
        var = ["n"],
        out = ["max_angle"]
    )
    >> gr.cp_bounds(
        n = (0.5, 3)
    )
)

In [12]:
(
    md_minimize
    >> gr.ev_min(
        out_min = "max_angle"
    )
)

,n,n_0,max_angle,success,message,n_iter
0,1.75,1.75,0.15738,True,Optimization terminated successfully,1


In [ ]:
(
df_hull
    >> ggplot(aes("x", "y"))
    + geom_point()
    + theme_minimal()
    + coord_fixed()
)

In [ ]:
# for n in np.linspace(0.5, 3, 6):
#     max_angle, df_sim = kaboodle(n)
#     print(max_angle)

Run once and get max angle

In [ ]:
df_sim['theta_b'].plot(label = "boat")
# df_sim['moment'].plot(label = "moment")
ts = df_sim.index.tolist()
ws=[]
for t in ts: 
    ws.append(w_func(t))
plt.plot(ts, ws, label = "water")
plt.legend()
# plt.ylim(-1, 5)